In [1]:
import pandas as pd

In [2]:
tables = pd.read_html("https://en.wikipedia.org/wiki/2020_United_States_presidential_election")

In [3]:
tables = [x for x in tables if "Biden/Harris" in str(x)]

In [4]:
table = tables[-1]
table = table.set_index(("State ordistrict", "State ordistrict"))

In [5]:
table

Biden/HarrisDemocratic  \
                                                       Votes   
(State ordistrict, State ordistrict)                           
Alab.                                                 849624   
Alaska                                                153778   
Arizona                                              1672143   
Ark.                                                  423932   
Calif.                                              11110250   
Colo.                                                1804352   
Conn.                                                1080831   
Del.                                                  296268   
D.C.                                                  317323   
Florida                                              5297045   
Georgia                                              2473633   
Hawaii                                                366130   
Idaho                                                 287021   
Illinois                                             3471915   
Indiana                                              1242416   
Iowa                                                  759061   
Kansas                                                570323   
Ky.                                                   772474   
La.                                                   856034   
Maine †                                               435072   
ME-1                                                  266376   
ME-2                                                  168696   
Md.                                                  1985023   
Mass.                                                2382202   
Mich.                                                2804040   
Minn.                                                1717077   
Miss.                                                 539398   
Mo.                                                  1253014   
Mont.                                                 244786   
Neb. †                                                374583   
NE-1                                                  132261   
NE-2                                                  176468   
NE-3                                                   65854   
Nev.[p]                                               703486   
N.H.                                                  424937   
N.J.[q]                                              2608335   
N.M.                                                  501614   
N.Y.[r]                                              5230985   
N.C.                                                 2684292   
N.D.                                                  114902   
Ohio                                                 2679165   
Okla.                                                 503890   
Oregon                                               1340383   
Pa.                                                  3458229   
R.I.                                                  307486   
S.C.                                                 1091541   
S.D.                                                  150471   
Tenn.                                                1143711   
Texas[t]                                             5259126   
Utah                                                  560282   
Vt.                                                   242820   
Va.                                                  2413568   
Wash.                                                2369612   
W.Va.                                                 235984   
Wis.                                                 1630866   
Wyo.                                                   73491   
Total                                               81268924   
NaN                                   Biden/HarrisDemocratic   

                                                              \
                                                           %   
(Stat

In [6]:
evs = table[[("Biden/HarrisDemocratic", "EV"), ("Trump/PenceRepublican", "EV")]][
    :-2
].applymap(lambda x: int(x) if x != "–" else 0).T.sum()
evs = {k : v for k, v in zip(evs.index, evs) if "-" not in k}
evs["Maine †"] += 2
evs["Neb. †"] += 3
del evs["D.C."]
cds = {k : v - 2 for k, v in evs.items()}

In [7]:
sum(cds.values())

435

In [8]:
margins = {
    st: (float(table.loc[st, ("Margin", "%")].replace("%", "").replace("−", "-")))
    / 100
    for st in cds
}

In [9]:
def amount(count, dem_voteshare):
    dem = count * dem_voteshare
    dem = round(dem)
    dem = max(dem, 0)
    dem = min(dem, count)
    return dem

def proportionality(count, margin):
    return amount(count, 0.5 + margin / 2)

def efficiency_gap(count, margin):
    return amount(count, 0.5 + margin)

In [10]:
dem_prop = {st : proportionality(cds[st], margins[st]) for st in margins}
sum(dem_prop.values())

225

In [11]:
dem_eff = {st : efficiency_gap(cds[st], margins[st]) for st in margins}
sum(dem_eff.values())

236

In [12]:
proportionality(52, margins["Calif."])

34

In [13]:
frame = pd.DataFrame({"Proportionality" : dem_prop, "Efficiency Gap" : dem_eff})
frame["Difference"] = frame["Efficiency Gap"] - frame["Proportionality"]

In [14]:
frame.sort_values("Difference")

,Proportionality,Efficiency Gap,Difference
Alab.,3,2,-1
Ky.,2,1,-1
Kansas,2,1,-1
Indiana,4,3,-1
Miss.,2,1,-1
Idaho,1,0,-1
Okla.,2,1,-1
Tenn.,3,2,-1
Texas[t],17,16,-1
Utah,2,1,-1
